# Early OTE Commissioning Guiding PSF Analysis

During the five non-deployment OTE-CARs that precede LOS-02, the WF Guiding team will do periodic checks of the PSFs and determine if there are any concerns for guiding in LOS-02. This notebook will walk through image analysis of images from OTE-01, OTE-02, OTE-03, OTE-04, and OTE-06 AFTER they have been executed. We do not expect analysis of OTE-05 to be necessary. We focus on PSF shapes and how many "guideable" segment PSFs we have at each stage. 

It is important to note that this notebook will do very similar analysis for all CARs, with some small adjustments for OTE-03 and OTE-04 because of the type of data we will get. A lot of the data is presented without drawing conclusions, it will be up to you to do so. In some cases you might find the there is more information than you think is helpful, it is up to you to determine what is helpful and what is not. 

As each step is completed, as needed, please communicate a summary of what you have found to WF Ops via email or over the loops during a non-busy time in commissioning. 

*It is very important to provide WF Ops with an update if you think the number of "guidable" segment PSFs is less than 6 prior to OTE-03.*

### To save a plot

If you want to save any plots to be printed or just viewed in Preview or another window, right click on the plots you want to save and then choose "Save image as..."

In [ ]:
# Make sure to run this cell before moving to one of the analysis sections
import os
from glob import glob

from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd

import notebook_utils
from notebook_utils import GA_PSF_LOCATIONS

font = {'size' : 16}
matplotlib.rc('font', **font)

%matplotlib inline

# Table of Contents
1. [Overview of products from Shadow](#Overview-of-the-products-we-can-expect-from-Shadow-and-others)
2. [After OTE-01](#After-OTE-01)
3. [After OTE-02](#After-OTE-02)
4. [After OTE-03](#After-OTE-03)
5. [After OTE-04](#After-OTE-04)
6. [After OTE-05](#After-OTE-05) - no WF Guiding analysis currently expected
7. [After OTE-06](#After-OTE-06)

The following sections are support sections and may not be run

8. [Examine all OTE-03 Data](#Look-at-different-focus-positions-in-OTE-03-data)
9. [Grab Large Image Array PSF Locations](#Get-simulated-NIRCam-locations-of-LOS-02-PSFs)


# Overview of the products we can expect from Shadow and others 

### What you will expect to have from Shadow
1. Path to the mosaic or knowledge about which file(s) to look for if they are coming via DANs
2. (Optional) A general idea of the location of the target star in the mosaic image
3. When segment information is known, a location where we can find the printout/electronic copy of which segment is which

# After OTE 01 

## OTE-01: Initial Image Mosiac

OTE-01 is the initial image mosaic and its goal is to find the 18 segments. 

There will be a range of focus for the PSFs; some will be close to focus, some will be out of focus. This will cause several of the segment PSFs to be saturated because, by requirement, the target star should be very bright and isolated. If a PSF is not saturated, it likely has a large defocus. The closer to focus, the more saturation will expected. 

After finding the segments, either Shadow or WF Guiding, through this notebook, will define an initial offset of each segment PSF from the target location, which is given by a rough estimate of the center of the PMSA deployment. If the target PSFs are too bright, we can inspect the shape of the PSFs by looking at the fainter background stars, however this notebook is not currently set up for that path since the simulated images that we have do not have such an option.

For OTE-01, we will have some idea of the shape of the segment PSFs but no idea of any of the segments' identifications. 

**Note: Shadow downsamples the detector images for the mosaic** but we do have the normal DMS images, so Shadow can give you the name of the DMS file that has the segments for a not downsampled option. If that is the case, look at the [OTE-03](#After-OTE-03) or [OTE-04](#After-OTE-04) sections to know how to combine the detector images to make our own mosaic.

#### After OTE-01, Shadow will provide:
 - End of OTE-01 mosaic path (will be available in the `/data/...` directory in SOGS)


<font color='Tomato'> <div><center>  
## Make sure the following parameters are correct
</center></div></font>

In [ ]:
# CAR information
car = 'OTE-01' # Make sure this form matches how it is written in the path to the data
program_id = 1134

In [ ]:
# Directory where the files can be found
main_directory = '/path/to/data/'
# Path to the OTE-01 mosaic
ote01_mosaic = os.path.join(main_directory, f'{car}/mosaic0_obs1-2_median_sub_gbrady_05032021.fits')

# The original mosaic will be resampled, see if shadow can give you the new pixelscale
ote01_pixelscale = None # If set to None, the NIRCam SW pixelscale of 0.031"/pixel will be used

# Define  the (x, y) pixel location of the target in the image
# If no location is provided, set variable equal to None and the code will take the median of the PSF locations
ote01_target_location = None 

<font color='DodgerBlue'> <div><center>  
## Run the following cells without changing them   
</center></div></font>

The below cell will take a little while to run because the image is large. 

In [ ]:
# We use the header for getting the pixel scale so it's okay that it is for only one of the images
ote01_image, ote01_header = fits.getdata(ote01_mosaic, header=True)

# Create a PsfAnalysis object for OTE-02, this is how we can do most of the analysis
ote01 = notebook_utils.PsfAnalysis(car, ote01_image, ote01_header, ote01_pixelscale, ote01_target_location)

In [ ]:
# Measure the PSF locations and their characteristics for the OTE-01 mosaic
# Adjust the psf_window_radius if the window around the cut out PSFs below as needed
# This function will measure the FWHM_x, FWHM_y, the distance to the target location, 
#  and the encircled energy for each PSF
ote01.get_image_information(smoothing='high', psf_window_radius=40)

In [ ]:
# Plot the full mosaic with the estimated target location and identified PSFs
ote01.plot_image_with_psfs(xlim=None, # If you need to change the x axis limits, do so here
                           ylim=None, # If you need to change the y axis limits, do so here
                           label_color='white') # If you can't read the PSF labels, change this parameter

<font color='SlateBlue'> <div>

### Things to look for:    
 
</div></font>


- How many segment PSFs are saturated (zeros in the center of the PSF)?
- Is there a single bright spot in the PSF or are there multiple knots/lobes?


In [ ]:
# Cut out each segment and do a visual check
ote01.plot_each_identified_segment_psf(num_psf_per_row=4,
                                       labels=None) # Change this if you have segment knowledge

IMPORTANT NOTE: Any PSFs that have saturated cores may not have reliable FWHM or EE measurements

In [ ]:
# Let's look at the measured properties of each PSF
# The index on the left is the same as the numbering in the plots above
ote01.info_df

In the table, the x, y location of each PSF has been found, it's associated FWHM_x and FWHM_y have been measured, as well as the distance to the estimated target location.

## Look at FWHM values and distance to the target

In the plots below, a circle is plotted for each located PSF in the mosiac image. The color of the circle is indicated by the value of the parameter indicated in the title. Green indicates a better value, red indicates a worse value, relative to the other data points. 

The distance to target can tell us how much the PSF might change once it is moved to its large image array location

In [ ]:
# Look at FWHM x and y (though the images should tell you the same information)
ote01.plot_multiple_parameters(['fwhm_x', 'fwhm_y', 'distance_to_target'])

### If EE has been measured, plot it!

In [ ]:
# We will only measure EE if pixel scale is not None
# All plots have the SAME y axis
ote01.plot_ee(num_psf_per_row=4, 
              labels=None) # Change this if you have segment knowledge     

### Based on the above information, how many segment PSFs are good enough to guide one?

<font color='Tomato'> <div><center>    
### Update the next cell (only) as needed   
</center></div></font>

In [ ]:
# Give the number of PSFs that you think are good enough to guide on
good = 13 

If the number of "Good" PSFs is more than, or equal to 6, then we can feel confident that we can get a guiding scenario. If it is less than 6, then we will need OTE-03

In [ ]:
if good < 6:
    print(f'We only have {good} PSFs that we think are good enough for guiding. We need more than 6.')
else:
    print(f'We have {good} PSFs that we think are good enough for guiding, which might be enough.')

# After OTE 02

## OTE-02: Alignment Matrix 1

WSS/WSS Shadow will use the OTE-01 initial mosaic offset from the expected target to do the first telescope correction for OTE-02, so that it points roughy to the center of the initial deployment. 

For OTE-02 we have NIRCam ALL images, so we should have a partial view of the initial deployment (not mosaic) but these should have few, if any, saturated PSFs. Saturation in any of the PSFs will imply that the defocus is too large between all segments. 

The process for OTE-01 and OTE-02  for WF Guiding will be the same. Both OTE-01 and OTE-02 images are expected to have high backgrounds.

*It is very important to provide WF Ops with an update if you think the number of "guidable" segment PSFs is less than 6 prior to OTE-03.*

#### After OTE-02, Shadow will provide:
 - End of OTE-02 mosaic path - if no mosaic is provided use the code shown in section [OTE-03](#After-OTE-03) to make your own
 - Any information from any discussions regarding PSF quality dicussed in the WF office **if time allows**

*Do the same analysis that you did after OTE-01, just with the image of the partial deployment*

<font color='Tomato'> <div><center>  
## Make sure the following parameters are correct
</center></div></font>

In [ ]:
# CAR information
car = 'OTE-02' # Make sure this form matches how it is written in the path to the data
program_id = 1135

In [ ]:
# Directory where the files can be found
main_directory = '/path/to/data/'
# Location of the OTE-02 mosaic with most segments
ote02_mosaic = os.path.join(main_directory, f'{car}/output/mosaic0_OTE-02-Obs1+2-SW.fits')

# Pixelscale of the image, replace the None below if the mosaic has been rebinned by shadow
ote02_pixelscale = None # If set to None, the NIRCam SW pixelscale of 0.031"/pixel will be used

# Define  the (x, y) pixel location of the target in the image
# If no location is provided, set variable equal to None
ote02_target_location = None

<font color='DodgerBlue'> <div><center>
## Run the following cells without changing them 
</center></div></font>

The below cell will take a little while to run because the image is large. 

In [ ]:
# We use the header for getting the pixel scale so it's okay that it is for only one of the images
ote02_image, ote02_header = fits.getdata(ote02_mosaic, header=True)

# Create a PsfAnalysis object for OTE-02, this is how we can do most of the analysis
ote02 = notebook_utils.PsfAnalysis(car, ote02_image, ote02_header, ote02_pixelscale, ote02_target_location)

In [ ]:
# Measure the PSF locations and their characteristics for the OTE-02 mosaic
# Adjust the psf_window_radius if the window around the cut out PSFs below as needed
# This function will measure the FWHM_x, FWHM_y, the distance to the target location, 
#  and the encircled energy for each PSF
ote02.get_image_information(smoothing='high', psf_window_radius=40)

In [ ]:
# Plot the full mosaic with the estimated target location and identified PSFs
ote02.plot_image_with_psfs(xlim=None, # If you need to change the x axis limits, do so here
                           ylim=None, # If you need to change the y axis limits, do so here
                           label_color='white') # If you can't read the PSF labels, change this parameter

<font color='SlateBlue'> <div>

### Things to look for:    
 
</div></font>


- How many segment PSFs are saturated (zeros in the center of the PSF)?
- Is there a single bright spot in the PSF or are there multiple knots/lobes?


In [ ]:
# Cut out each segment and do a visual check
ote02.plot_each_identified_segment_psf(num_psf_per_row=4,
                                       labels=None) # Change this if you have segment knowledge

IMPORTANT NOTE: Any PSFs that have saturated cores may not have reliable FWHM or EE measurements

In [ ]:
# Let's look at the properties of each PSF. 
# The index on the left is the same as the numbering in the plots above
ote02.info_df

In the table, the x, y location of each PSF has been found, it's associated FWHM_x and FWHM_y have been measured, as well as the distance to the estimated target location.

## Look at FWHM values and distance to the target

In the plots below, a circle is plotted for each located PSF in the mosiac image. The color of the circle is indicated by the value of the parameter indicated in the title. Green indicates a better value, red indicates a worse value, relative to the other data points. 

The distance to target can tell us how much the PSF might change once it is moved to its large image array location

In [ ]:
# Look at FWHM x and y (though the images should tell you the same information)
ote02.plot_multiple_parameters(['fwhm_x', 'fwhm_y', 'distance_to_target'])

### If EE has been measured, plot it!

In [ ]:
# We will only measure EE if pixel scale is not None
# All plots have the SAME y axis
ote02.plot_ee(num_psf_per_row=4, 
              labels=None) # Change this if you have segment knowledge     

### Based on the above information, how many segment PSFs are good enough to guide one?

<font color='Tomato'> <div><center>    
### Update the next cell (only) as needed   
</center></div></font>

In [ ]:
# Give the number of PSFs that you think are good enough to guide on
good = 13 

If the number of "Good" PSFs is more than, or equal to 6, then we can feel confident that we can get a guiding scenario. If it is less than 6, then we will need OTE-03

In [ ]:
if good < 6:
    print(f'We only have {good} PSFs that we think are good enough for guiding. We need more than 6.')
else:
    print(f'We have {good} PSFs that we think are good enough for guiding, which might be enough.')

# After OTE 03

## OTE-03: Secondary Mirror Focus Sweep 

**OTE-03 is optional, but likely to occur.** 

During OTE-03, the secondary mirror will be moved 100um in either direction in order to find the best focus for the segment PSFs that are visible in the field.

We will have the NRCALL exposures, as with OTE-02, with as many segments in the field as possible. At the end of OTE-03 we might have a tentative segment ID (though we will not consider this in our analysis) and should have better (or more compact) PSFs for guiding.

WF Guiding should have recieved DANs for all OTE-03 data.

#### After OTE-03, Shadow will provide:
 - Any idea regarding which segment PSF is which. There will be a formal list somewhere, in a PNG (will be avilable on a pring out or in `/data/…`)


If you want to just inspect the images with the secondary mirror at different focus positions, you can do this [here](#Look-at-different-focus-positions-in-OTE-03-data)

<font color='Tomato'> <div><center>  
## Make sure the following parameters are correct
</center></div></font>

In [ ]:
# CAR information
car = 'OTE-03' # Make sure this form matches how it is written in the path to the data
program_id = 1136

In [ ]:
# Directory where the files can be found
main_directory = '/path/to/data/'

# Location of the NIRCam images at the best focus
ote03_best_focus_images = glob(os.path.join(main_directory, 
                                            f'{car}/output/jw01136003001_01101_00002_nrc*_cal.fits'))

# Pixelscale of the image, replace the None below if the mosaic has been rebinned by shadow
ote03_pixelscale = None # If set to None, the NIRCam SW pixelscale of 0.031"/pixel will be used

# Define  the (x, y) pixel location of the target in the image
# If no location is provided, set variable equal to None
ote03_target_location = None

In [ ]:
ote03_best_focus_images

<font color='DodgerBlue'> <div><center>  
## Run the following cells without changing them 
</center></div></font>

Now, we have a better idea of which segment is which, AND we have focus diversity. Here we will be able to match a guess of the PSF in the images we have with their segment name, and then cut that out of the images that we have and put at their GA positions. This prepares us for OTE-07.

In [ ]:
# Plot out all all nircam images, and make our own mosiac

# -------------------     -------------------
# |        |        |     |        |        | 
# |   A2   |   A4   |     |   B3   |   B1   |
# |        |        |     |        |        |
# |        |        |     |        |        |
# -------------------     -------------------
# |        |        |     |        |        |
# |   A1   |   A3   |     |   B4   |   B2   |
# |        |        |     |        |        |
# |________|________|     |________|________| 

In [ ]:
ote03_nrca_images, ote03_nrcb_images = notebook_utils.separate_nircam_images(ote03_best_focus_images)

nrca_data_list, nrca_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrca_images)
notebook_utils.plot_nrca_images(nrca_data_list, nrca_name_list)

nrcb_data_list, nrcb_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrcb_images)
notebook_utils.plot_nrcb_images(nrcb_data_list, nrcb_name_list)

In [ ]:
# Make our mosaic
ote03_image = notebook_utils.create_basic_mosaic(nrca_data_list, nrcb_data_list)

plt.figure(figsize=(10,20))
plt.imshow(ote03_image, norm=LogNorm(vmin=1, vmax=1000), origin='lower')
plt.title(f'{car} mosaic')
plt.axis('off')
plt.show()

<font color='DodgerBlue'> <div><center>
## Run the following cells without changing them 
</center></div></font>

In [ ]:
# We use the header for getting the pixel scale so it's okay that it is for only one of the images
ote03_header = fits.getheader(ote03_best_focus_images[0]) 

# Create a PsfAnalysis object for OTE-03, this is how we can do most of the analysis
ote03 = notebook_utils.PsfAnalysis(car, ote03_image, ote03_header, ote03_pixelscale, ote03_target_location)

In [ ]:
# Measure the PSF locations and their characteristics for the OTE-03 image that we have
# Adjust the psf_window_radius if the window around the cut out PSFs below as needed
# This function will measure the FWHM_x, FWHM_y, the distance to the target location, 
#  and the encircled energy for each PSF
ote03.get_image_information(smoothing='high', psf_window_radius=100)

In [ ]:
# Plot the full mosaic with the estimated target location and identified PSFs
ote03.plot_image_with_psfs(xlim=None, # If you need to change the x axis limits, do so here
                           ylim=None, # If you need to change the y axis limits, do so here
                           label_color='white') # If you can't read the PSF labels, change this parameter

<font color='SlateBlue'> <div>

### Things to look for:    
 
</div></font>


- How many segment PSFs are saturated (zeros in the center of the PSF)?
- Is there a single bright spot in the PSF or are there multiple knots/lobes?


In [ ]:
# Cut out each segment and do a visual check
ote03.plot_each_identified_segment_psf(num_psf_per_row=4,
                                       labels=None # Change this if you have segment knowledge
                                      )

IMPORTANT NOTE: Any PSFs that have saturated cores may not have reliable FWHM or EE measurements

In [ ]:
# Let's look at the properties of each PSF. 
# The index on the left is the same as the numbering in the plots above
ote03.info_df

In the table, the x, y location of each PSF has been found, it's associated FWHM_x and FWHM_y have been measured, as well as the distance to the estimated target location.

## Look at FWHM values and distance to the target

In the plots below, a circle is plotted for each located PSF in the mosiac image. The color of the circle is indicated by the value of the parameter indicated in the title. Green indicates a better value, red indicates a worse value, relative to the other data points. 

The distance to target can tell us how much the PSF might change once it is moved to its large image array location

In [ ]:
# Look at FWHM x and y (though the images should tell you the same information)
ote03.plot_multiple_parameters(['fwhm_x', 'fwhm_y', 'distance_to_target'])

### If EE has been measured, plot it!

In [ ]:
# We will only measure EE if pixel scale is not None
# All plots have the SAME y axis
ote03.plot_ee(num_psf_per_row=4, 
              labels=None) # Change this if you have segment knowledge     

### Based on the above information, how many segment PSFs are good enough to guide one?

<font color='Tomato'> <div><center>    
### Update the next cell (only) as needed   
</center></div></font>

In [ ]:
# Give the number of PSFs that you think are good enough to guide on
good = 13 

If the number of "Good" PSFs is more than, or equal to 6, then we can feel confident that we can get a guiding scenario. If it is less than 6, then we will need OTE-03

In [ ]:
if good < 6:
    print(f'We only have {good} PSFs that we think are good enough for guiding. We need more than 6.')
else:
    print(f'We have {good} PSFs that we think are good enough for guiding, which might be enough.')

# After OTE 04

## OTE-04: Segment ID

At the end of OTE-04 we should have the ID of each segment, with images of all the segment PSFs. In order to find all the PSFs, there will likely be multiple pointings so that we can image and identify all PSFs. It's likely that we will need to analyze a combination of multiple pointings in order to complete our analysis of all segment PSFs. A possible caveat is that some segments may be on top or close to each other, thus, some contamination could affect the PSF measurements.

As this point, WF Guiding should be able to make a fake large image array image that can be run through MAGIC and DHAS. All successful guide/reference star configurations, should be recorded.

#### After OTE-04, Shadow will provide:
 - Which segment is which in either an OTE-04 image printout or online. Format and communication is unclear (will be avilable on a print out or in `/data/…`)


**Ask Shadow which sets of pointings will have all of the segments and only load those images.**

<font color='Tomato'> <div><center>  
## Make sure the following parameters are correct
</center></div></font>

In [ ]:
# CAR information
car = 'OTE-04' # Make sure this form matches how it is written in the path to the data
program_id = 1137

In [ ]:
# Directory where the files can be found
main_directory = '/path/to/data/'

# Pixelscale of the image, replace the None below if the mosaic has been rebinned by shadow
ote04_pixelscale = None # If set to None, the NIRCam SW pixelscale of 0.031"/pixel will be used

# Pointing 1
# Choose the best set of exposures for the first pointing
ote04_pointing1_images = glob(os.path.join(main_directory, 
                                           f'{car}/output/jw01137001001_01101_00019_nrc*_cal.fits'))

# Define the (x, y) pixel location of the target in the image
# If no location is provided, set variable equal to None
ote04_target_location1 = None


# Pointing 2
# Choose the best set of exposures for the first pointing
ote04_pointing2_images = glob(os.path.join(main_directory, 
                                           f'{car}/output/jw01137002001_01101_00004_nrc*_cal.fits'))

# Define the (x, y) pixel location of the target in the image
# If no location is provided, set variable equal to None
ote04_target_location2 = None

# Pointing 3
# Include more pointings as they exist.
# ote04_pointing3 = glob(os.path.join(main_directory, 
#                                     f'{car}/output/jw01137003001_01101_00004_nrc*_cal.fits'))


ote04_matching_dictionary = {}

<font color='DodgerBlue'> <div><center>  
## Run the following cells without changing them 
</center></div></font>

Now, we know which segment is which. After OTE-04 will be the first time that we will be able to create the full large image array configuration that can be saved to an image to be run through MAGIC and DHAS. 

To do this, we will first make separate mosaic images for each pointing, and then, using the provided information about which PSF is which segment, we can cut out those PSFs from their respective mosaic images and create a large image array configuration.

In [ ]:
# Plot out all all nircam images, and make our own mosiac

# -------------------     -------------------
# |        |        |     |        |        | 
# |   A2   |   A4   |     |   B3   |   B1   |
# |        |        |     |        |        |
# |        |        |     |        |        |
# -------------------     -------------------
# |        |        |     |        |        |
# |   A1   |   A3   |     |   B4   |   B2   |
# |        |        |     |        |        |
# |________|________|     |________|________| 

### Pointing 1 

In [ ]:
ote04_pointing1_images

In [ ]:
ote04_nrca_images1, ote04_nrcb_images1 = notebook_utils.separate_nircam_images(ote04_pointing1_images)

nrca_data_list1, nrca_name_list1 = notebook_utils.get_nrc_data_from_list(ote04_nrca_images1)
# If you want to plot out the NRCA images, uncomment the below line
# notebook_utils.plot_nrca_images(nrca_data_list1, nrca_name_list1)

nrcb_data_list1, nrcb_name_list1 = notebook_utils.get_nrc_data_from_list(ote04_nrcb_images1)
# If you want to plot out the NRCB images, uncomment the below line
# notebook_utils.plot_nrcb_images(nrcb_data_list1, nrcb_name_list1)

# Make our mosaic
ote04_image1 = notebook_utils.create_basic_mosaic(nrca_data_list1, nrcb_data_list1)

In [ ]:
# We use the header for getting the pixel scale so it's okay that it is for only one of the images
ote04_header = fits.getheader(ote04_pointing1_images[0]) 

# Create a PsfAnalysis object for OTE-03, this is how we can do most of the analysis
ote04_pointing1 = notebook_utils.PsfAnalysis(car, ote04_image1, ote04_header, ote04_pixelscale, 
                                             ote04_target_location1)

# Measure the PSF locations and their characteristics for the OTE-03 image that we have
# Adjust the psf_window_radius if the window around the cut out PSFs below as needed
ote04_pointing1.get_image_information(smoothing='high', psf_window_radius=100)

In [ ]:
# Plot the full mosaic with the estimated target location and identified PSFs
ote04_pointing1.plot_image_with_psfs(xlim=None, # If you need to change the x axis limits, do so here
                                     ylim=None, # If you need to change the y axis limits, do so here
                                     label_color='white', # If you can't read the PSF labels, change this parameter
                                     legend_location='lower left' # 'best', 'upper left', 'upper right', 'lower left', 'lower right'
                                     )                   # 'upper center', 'lower center', 'center left', 'center right' 

In [ ]:
# With the help of Shadow, determine the segment ID, if known, for each of the labelled PSFs in the above image 
# In the dictionary below, for each known PSF, record the associated number: segment ID. 

# Segment names: A1-1, A2-2, A3-3, A4-4, A5-5, A6-6, 
#                B1-7, B2-9, B3-11, B4-13, B5-15, B6-17, 
#                C1-8, C2-10, C3-13, C4-14, C5-16, C6-18

# Delete any entry where that segment is not present in the above image.
# Do not include partial segments unless this is the only image that they are in
# DO NOT include any repeats
ote04_matching_dictionary['pointing_1'] = {7  :'A1-1', 
                                            6  :'A3-3', 
                                            0  :'A4-4', 
                                            4  :'A5-5', 
                                            10 :'A6-6', 
                                            9  :'B1-7',
                                            13 :'B2-9', 
                                            14 :'B3-11', 
                                            1  :'B4-13', 
                                            2  :'B5-15', 
                                             3 :'C3-12', 
                                            11 :'C4-14', 
                                          }

### Pointing 2 

In [ ]:
ote04_pointing2_images

In [ ]:
ote04_nrca_images2, ote04_nrcb_images2 = notebook_utils.separate_nircam_images(ote04_pointing2_images)

nrca_data_list2, nrca_name_list2 = notebook_utils.get_nrc_data_from_list(ote04_nrca_images2)
# If you want to plot out the NRCA images, uncomment the below line
# notebook_utils.plot_nrca_images(nrca_data_list2, nrca_name_list2)

nrcb_data_list2, nrcb_name_list2 = notebook_utils.get_nrc_data_from_list(ote04_nrcb_images2)
# If you want to plot out the NRCB images, uncomment the below line
# notebook_utils.plot_nrcb_images(nrcb_data_list2, nrcb_name_list2)

# Make our mosaic
ote04_image2 = notebook_utils.create_basic_mosaic(nrca_data_list2, nrcb_data_list2)

In [ ]:
# We use the header for getting the pixel scale so it's okay that it is for only one of the images
ote04_header = fits.getheader(ote04_pointing2_images[0]) 

# Create a PsfAnalysis object for OTE-03, this is how we can do most of the analysis
ote04_pointing2 = notebook_utils.PsfAnalysis(car, ote04_image2, ote04_header, ote04_pixelscale, 
                                             ote04_target_location2)

# Measure the PSF locations and their characteristics for the OTE-03 image that we have
# Adjust the psf_window_radius if the window around the cut out PSFs below as needed
ote04_pointing2.get_image_information(smoothing='high', psf_window_radius=100)

In [ ]:
# Plot the full mosaic with the estimated target location and identified PSFs
ote04_pointing2.plot_image_with_psfs(xlim=None, # If you need to change the x axis limits, do so here
                                     ylim=None, # If you need to change the y axis limits, do so here
                                     label_color='white') # If you can't read the PSF labels, change this parameter

<font color='Tomato'> <div><center>    
## Make sure the following parameters are correct   
</center></div></font>

In [ ]:
# With the help of Shadow, determine the segment ID, if known, for each of the labelled PSFs in the above image 
# In the dictionary below, for each known PSF, record the associated number: segment ID. 

# Segment names: A1-1, A2-2, A3-3, A4-4, A5-5, A6-6, 
#                B1-7, B2-9, B3-11, B4-13, B5-15, B6-17, 
#                C1-8, C2-10, C3-13, C4-14, C5-16, C6-18

# Delete any entry where that segment is not present in the above image.
# Do not include partial segments unless this is the only image that they are in
# DO NOT include any repeats
ote04_matching_dictionary['pointing_2'] ={5   :'A2-2', 
                                          0   :'B6-17', 
                                          1   :'C1-8', 
                                          3   :'C2-10', 
                                          4   :'C5-16', 
                                          2   :'C6-18'
                                          }

### Combine the different pointings so that we can use all the data together

In [ ]:
ote04_combined = notebook_utils.PsfAnalysis(car, None, ote04_header, ote04_pixelscale, 
                                            None)

In [ ]:
info_dfs = [ote04_pointing1.info_df, ote04_pointing2.info_df]
ee_fn_lists = [ote04_pointing1.ee_fns, ote04_pointing2.ee_fns]
ote04_combined.info_df, ote04_combined.ee_fns = notebook_utils.create_combined_pointing_df(info_dfs, 
                                                                                           ote04_matching_dictionary,
                                                                                           ee_fn_lists)

ote04_combined.info_df

In [ ]:
# Put the images in order of the pointings
pointings_images = [ote04_pointing1.corrected_image, ote04_pointing2.corrected_image]
psf_window_radius = 100
ote04_combined.psfs = notebook_utils.get_psfs_from_all_pointings(ote04_combined.info_df, pointings_images, psf_window_radius)
ote04_combined.psf_window_radius = psf_window_radius

In [ ]:
# Cut out each segment and do a visual check
# Change the size of the postage stamp cut out, by adjusting the window_size value which is the size of one side
ote04_combined.plot_each_identified_segment_psf(num_psf_per_row=4,
                                                labels=ote04_combined.info_df['segment'].values)

### Place the segments at their Large Image Array configuration

In [ ]:
# Put known segment postage stamps in large image array locations
ote04_large_image_array, ga_xs, ga_ys = notebook_utils.create_image_array(ote04_combined.psfs,
                                                                          ote04_combined.info_df['segment'].values,
                                                                          ote04_combined.psf_window_radius)

# Plot out new image with distance color coded by how far from boresight
plt.figure(figsize=(10, 10))
plt.imshow(ote04_large_image_array, norm=LogNorm(vmin=1), origin='upper')
plt.title(f"Assumed Large Image Array created from {car} data")
plt.colorbar()
plt.show()

In [ ]:
# Save this out to an image that we can run through MAGIC
out_dir = '~/Desktop/' # Make sure this is where you want to save it
filename = f'pseudo_large_image_array_from_{car}.fits'
notebook_utils.save_out_image_for_magic(ote04_large_image_array, ote04_header, filename, out_dir)

### Plot heat map of distances from estimated target location 

Here we are showing how much a single segment PSF will have to move to get it near to the target location. This implies that the shape of the PSF might change once it is put in the large image array configuration. 

In [ ]:
# Look at high level numbers
ote04_combined.plot_multiple_parameters(['distance_to_target'], xs=ga_xs, ys=ga_ys,
                                        xlim=(0,2048), ylim=(2048, 0))

In [ ]:
# Break down the FWHM further
ote04_combined.plot_multiple_parameters(['fwhm_x', 'fwhm_y'], xs=ga_xs, ys=ga_ys,
                                        xlim=(0, 2048), ylim=(2048, 0))

In [ ]:
# We will only measure EE if pixel scale is not None
# All plots have the SAME y axis
ote04_combined.plot_ee(num_psf_per_row=4, 
                       labels=ote04_combined.info_df['segment'].values)    

You can now run this file through MAGIC and see if you can find some selections that can be used for LOS-02

# After OTE 05

## OTE-05: Alignment Matrix 2

OTE-05 is similar to OTE-02.

We do not expect to have any new information from OTE-05 that can help WF Guiding prepare for LOS-02, so no analysis will be done. 

# After OTE 06

## OTE-06: Segment Image Array 1

During OTE-06, WSS forms the large array in NIRCam A3. At the end of OTE-06 we'll have the same configuration and PSF as in LOS-02 and the first exposure of OTE-07. This is also a good time to do a full PSF characterization in preparation for Guiding.

#### After OTE-06, Shadow will provide:
- Nothing

*Do a quick run with the same analysis that you did after OTE-01, OTE-02, OTE-03, OTE-04.*

Then you will run the final image from OTE-06 through MAGIC as patrt of the nominal special guiding process

<font color='Tomato'> <div><center>  
## Make sure the following parameters are correct
</center></div></font>

In [ ]:
# CAR information
car = 'OTE-06' # Make sure this form matches how it is written in the path to the data
program_id = 1140

In [ ]:
# Directory where the files can be found
main_directory = '/path/to/data/'

# Location of the final image from OTE-06
ote06_file = os.path.join(main_directory, f'{car}/output/jw01140003001_01101_00019_nrca3_cal.fits')

# Pixelscale of the image, replace the None below if the mosaic has been rebinned by shadow
ote06_pixelscale = None # If set to None, the NIRCam SW pixelscale of 0.031"/pixel will be used

<font color='DodgerBlue'> <div><center>
## Run the following cells without changing them 
</center></div></font>

In [ ]:
# We use the header for getting the pixel scale
ote06_image, ote06_header = fits.getdata(ote06_file, header=True)

# Create a PsfAnalysis object for OTE-06, this is how we can do most of the analysis
ote06 = notebook_utils.PsfAnalysis(car, ote06_image, ote06_header, ote06_pixelscale)

In [ ]:
# Measure the PSF locations and their characteristics for the OTE-06 image that we have
# Adjust the psf_window_radius if the window around the cut out PSFs below as needed
ote06.get_image_information(smoothing='high', psf_window_radius=40)

In [ ]:
# Plot the image with the estimated target location (center of pointing) and identified PSFs
ote06.plot_image_with_psfs(xlim=None, # If you need to change the x axis limits, do so here
                           ylim=None, # If you need to change the y axis limits, do so here
                           label_color='white') # If you can't read the PSF labels, change this parameter

<font color='SlateBlue'> <div>

### Things to look for:    
 
</div></font>


- How many segment PSFs are saturated (zeros in the center of the PSF)?
- Is there a single bright spot in the PSF or are there multiple knots/lobes?


In [ ]:
# Cut out each segment and do a visual check
ote06.plot_each_identified_segment_psf(num_psf_per_row=4,
                                       labels=None) # Change this if you have segment knowledge

IMPORTANT NOTE: Any PSFs that have saturated cores may not have reliable FWHM or EE measurements

In [ ]:
# Let's look at the properties of each PSF. 
# The index on the left is the same as the numbering in the plots above
ote06.info_df

In the table, the x, y location of each PSF has been found, it's associated FWHM_x and FWHM_y have been measured, as well as the distance to the estimated target location.

## Look at FWHM values and distance to the target

In the plots below, a circle is plotted for each located PSF in the mosiac image. The color of the circle is indicated by the value of the parameter indicated in the title. Green indicates a better value, red indicates a worse value, relative to the other data points. 

In [ ]:
# Look at FWHM x and y (though the images should tell you the same information)
ote06.plot_multiple_parameters(['fwhm_x', 'fwhm_y', 'distance_to_target'])

### If EE has been measured, plot it!

In [ ]:
# We will only measure EE if pixel scale is not None
# All plots have the SAME y axis
ote06.plot_ee(num_psf_per_row=4, 
              labels=None) # Change this if you have segment knowledge     

### Congrats! You have made it to the start of the special guiding part of OTE Commissioning! Good Luck!

Below you will find some additional support analysis that is optional. 

# Look at different focus positions in OTE-03 data

This section is set up so that you can visually inspect the OTE-03 images as the secondary mirror moves through the different focus positions.

This section has been built off my (K. Brooks) understanding of OTE-03, that the first image in the observation will be without a mirror move and that the subsequent exposures in the observation will be taken at the 5 focus sweep positions. Please confirm this process, as well as the secondary mirror locations during the sweep (since this is will be determined in flight) with Shadow before running the following cells. 

In [ ]:
# CAR information
car = 'OTE-03'  # Make sure this form matches how it is written in the path to the data
program_id = 1136

In [ ]:
# Directory where the files can be found
main_directory = '/path/to/data/'

# Location of the NIRCam images at the best focus
ote03_best_focus_images = glob(os.path.join(main_directory, 
                                            f'{car}/output/jw01136003001_01101_00002_nrc*_cal.fits'))


# Location of the file exported from QUIP
# If no file is provided, set variable equal to None
ote03_info_from_shadow = None

# Define location of the target in the image
# If no location is provided, set variable equal to None
ote03_target_location = None

In [ ]:
# Only grab shortwave detector images
ote03_focus1_images = glob(os.path.join(main_directory, 
                                        f'{car}/output/jw01136001001_01101_00001_nrc*[1-4]_cal.fits'))

ote03_focus2_images = glob(os.path.join(main_directory, 
                                        f'{car}/output/jw01136001001_01101_00002_nrc*[1-4]_cal.fits'))

ote03_focus3_images = glob(os.path.join(main_directory, 
                                        f'{car}/output/jw01136001001_01101_00003_nrc*[1-4]_cal.fits'))

ote03_focus4_images = glob(os.path.join(main_directory, 
                                        f'{car}/output/jw01136001001_01101_00004_nrc*[1-4]_cal.fits'))

ote03_focus5_images = glob(os.path.join(main_directory, 
                                        f'{car}/output/jw01136001001_01101_00005_nrc*[1-4]_cal.fits'))

ote03_focus6_images = glob(os.path.join(main_directory, 
                                        f'{car}/output/jw01136001001_01101_00006_nrc*[1-4]_cal.fits'))

# The defocus positions used are not set in stone and different positions may be used 
#  depending on what is determined by WSS. Ask Shadow for the defocus positions if this is
#  not known to you via WF room communications
sm_defocus_positions = [-400, -200, 0, 200, 400]

<font color='DodgerBlue'> <div><center>  
## Run the following cells without changing them 
</center></div></font>

In [ ]:
ote03_nrca_images, ote03_nrcb_images = notebook_utils.separate_nircam_images(ote03_focus1_images)
# Separate out the data
nrca_data_list, nrca_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrca_images)
nrcb_data_list, nrcb_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrcb_images)

# Make our mosaic
ote03_focus1 = notebook_utils.create_basic_mosaic(nrca_data_list, nrcb_data_list)

plt.figure(figsize=(10,20))
plt.imshow(ote03_focus1, norm=LogNorm(vmin=1, vmax=1000), origin='lower')
plt.title(f'{car} mosaic - 1st focus position, starting focus')
plt.axis('off')
plt.show()

In [ ]:
ote03_nrca_images, ote03_nrcb_images = notebook_utils.separate_nircam_images(ote03_focus2_images)
# Separate out the data
nrca_data_list, nrca_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrca_images)
nrcb_data_list, nrcb_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrcb_images)

# Make our mosaic
ote03_focus2 = notebook_utils.create_basic_mosaic(nrca_data_list, nrcb_data_list)

plt.figure(figsize=(10,20))
plt.imshow(ote03_focus2, norm=LogNorm(vmin=1, vmax=1000), origin='lower')
plt.title(f'{car} mosaic - 2st focus position: {sm_defocus_positions[0]}um')
plt.axis('off')
plt.show()

In [ ]:
ote03_nrca_images, ote03_nrcb_images = notebook_utils.separate_nircam_images(ote03_focus3_images)
# Separate out the data
nrca_data_list, nrca_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrca_images)
nrcb_data_list, nrcb_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrcb_images)

# Make our mosaic
ote03_focus3 = notebook_utils.create_basic_mosaic(nrca_data_list, nrcb_data_list)

plt.figure(figsize=(10,20))
plt.imshow(ote03_focus3, norm=LogNorm(vmin=1, vmax=1000), origin='lower')
plt.title(f'{car} mosaic - 3rd focus position: {sm_defocus_positions[1]}um')
plt.axis('off')
plt.show()

In [ ]:
ote03_nrca_images, ote03_nrcb_images = notebook_utils.separate_nircam_images(ote03_focus4_images)
# Separate out the data
nrca_data_list, nrca_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrca_images)
nrcb_data_list, nrcb_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrcb_images)

# Make our mosaic
ote03_focus4 = notebook_utils.create_basic_mosaic(nrca_data_list, nrcb_data_list)

plt.figure(figsize=(10,20))
plt.imshow(ote03_focus4, norm=LogNorm(vmin=1, vmax=1000), origin='lower')
plt.title(f'{car} mosaic - 4th focus position: {sm_defocus_positions[2]}um')
plt.axis('off')
plt.show()

In [ ]:
ote03_nrca_images, ote03_nrcb_images = notebook_utils.separate_nircam_images(ote03_focus5_images)
# Separate out the data
nrca_data_list, nrca_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrca_images)
nrcb_data_list, nrcb_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrcb_images)

# Make our mosaic
ote03_focus5 = notebook_utils.create_basic_mosaic(nrca_data_list, nrcb_data_list)

plt.figure(figsize=(10,20))
plt.imshow(ote03_focus5, norm=LogNorm(vmin=1, vmax=1000), origin='lower')
plt.title(f'{car} mosaic - 5th focus position: {sm_defocus_positions[3]}um')
plt.axis('off')
plt.show()

In [ ]:
ote03_nrca_images, ote03_nrcb_images = notebook_utils.separate_nircam_images(ote03_focus6_images)
# Separate out the data
nrca_data_list, nrca_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrca_images)
nrcb_data_list, nrcb_name_list = notebook_utils.get_nrc_data_from_list(ote03_nrcb_images)

# Make our mosaic
ote03_focus6 = notebook_utils.create_basic_mosaic(nrca_data_list, nrcb_data_list)

plt.figure(figsize=(10,20))
plt.imshow(ote03_focus6, norm=LogNorm(vmin=1, vmax=1000), origin='lower')
plt.title(f'{car} mosaic - 6th focus position: {sm_defocus_positions[4]}um')
plt.axis('off')
plt.show()

# Get simulated NIRCam locations of LOS-02 PSFs 

This section walks through getting the locations of each of the segment PSFs in the large image array configuration. This section is only to support the operations in OTE-04 above.

**Do not run these cells unless you need to find new locations of the segment PSFs in the large image array configuration**

In [ ]:
los02_image = fits.getdata('jw01410002001_02101_00001_nrca3_cal.fits')

In [ ]:
plt.figure(figsize=(10, 8))
plt.imshow(los02_image, norm=LogNorm(vmin=1))
plt.title('Large Image Configuration')
plt.show()

In [ ]:
# Do not change this cell
# mapping of magic segment labels to WSS numbers

SEGMENT_MAP_SCI = ["B1-7",  "C6-18", "C1-8",  "B6-17",
                   "A1-1",  "B2-9",  "A6-6",  "A2-2", 
                   "C5-16", "C2-10", "A5-5",  "A3-3", 
                   "B5-15", "A4-4",  "B3-11", "C4-14", 
                   "C3-12", "B4-13"]

In [ ]:
x_list, y_list = notebook_utils.get_position_from_magic(los02_image, 
                                                        smoothing='high', 
                                                        npeaks=np.inf)

In [ ]:
plt.figure(figsize=(10, 8))
plt.imshow(los02_image, norm=LogNorm(vmin=1))
for i, (seg, x, y) in enumerate(zip(SEGMENT_MAP_SCI, x_list, y_list)):
    plt.annotate(i, (x, y), (x+15, y+15), color='white' )
    plt.annotate(seg, (x, y), (x-30, y-30), color='white' )

In [ ]:
all_psfs_table = pd.DataFrame({'segment':SEGMENT_MAP_SCI, 'x':x_list, 'y':y_list, })

all_psfs_table

In [ ]:
truth_locations = all_psfs_table.set_index('segment').T.to_dict('list')
truth_locations

In [ ]:
truth_locations == notebook_utils.GA_PSF_LOCATIONS